In [5]:
from google.cloud import datastore
client = datastore.Client()


In [8]:
import datetime
region = 'sandiego_zone'
run_id = 'abc123'
model_type = 'zone'

def new_cloud_datastore_record(region, run_id, model_type):
    autospec_run = datastore.Entity(key=client.key('Autospec', run_id), exclude_from_indexes=('yaml',))
    autospec_run['created'] = datetime.datetime.utcnow()
    autospec_run['region'] = unicode(region)
    autospec_run['run_id'] = unicode(run_id)
    autospec_run['model_type'] = unicode(model_type)
    autospec_run['status'] = u'Initializing'
    autospec_run['progress'] = 0.0
    client.put(autospec_run)
    return autospec_run.key.id

In [9]:
new_cloud_datastore_record(region, run_id, model_type)

In [12]:
def record_progress(run_id, status, progress):
    key = client.key('Autospec', run_id)
    autospec_run = client.get(key)
    autospec_run['status'] = unicode(status)
    autospec_run['progress'] = progress
    client.put(autospec_run)

In [13]:
record_progress(run_id, 'Running', .2)

In [92]:
import yaml

import utils
import orca
import datasources
import variables

In [2]:
def get_selected_models_by_version(model_version_id):
    query = client.query(kind='Spec')
    query.add_filter('run_id', '=', model_version_id)
    query.add_filter('selected', '=', True)
    return list(query.fetch())

In [3]:
model_version_id = '70321032639rhx'
models = get_selected_models_by_version(model_version_id)

In [4]:
len(models)

10

In [95]:
model = models[0]

model.keys()

[u'segmentation',
 u'run_id',
 u'fit_filters',
 u'segment_id',
 u'dep_var',
 u'variables',
 u'tscores',
 u'r2',
 u'created',
 u'region',
 u'selected',
 u'observations_name',
 u'yaml',
 u'model_type',
 u'creator',
 u'model_name']

In [97]:
def make_lcm_func(model_name, yaml_file, agents_name, alts_name, alts_id_name, supply_attrib, remaining_capacity_attrib):
    @orca.step(model_name)
    def func():
        agents = orca.get_table(agents_name)
        alts = orca.get_table(alts_name)
        print yaml_file
        return utils.lcm_simulate(yaml_file, agents, alts,
                                  [],
                                  alts_id_name, supply_attrib,
                                  remaining_capacity_attrib)
    return func

def make_repm_func(model_name, yaml_file, dep_var, observations_name):
    """
    Generator function for block REPMs.
    """

    @orca.step(model_name)
    def func():
        obs = orca.get_table(observations_name)
        print yaml_file
        return utils.hedonic_simulate(yaml_file, obs,
                                      [], dep_var)
    return func

In [99]:
import yaml
for model in models:
    model_type = model['model_type']
    model_name = model['model_name']
    yaml_cfg = model['yaml']
    dict_cfg = yaml.load(yaml_cfg)
    
    if model_type == 'location':
        agents = model['agents']
        alternatives = model['alternatives']
        alternatives_id_name = dict_cfg['choice_column']
        supply_variable = agents + '_spaces'
        vacant_variable = 'vacant_' + supply_variable
        make_lcm_func(model_name,  yaml_cfg, agents, alternatives, alternatives_id_name, 
                      supply_variable, vacant_variable)

    elif model_type == 'regression':
        dep_var = model['dep_var']
        obs_name = model['observations_name']
        make_repm_func(model_name, yaml_cfg, dep_var, obs_name)
        
    else:
        print 'Model type "%s" unrecognized' % model_type

In [82]:
[u'segmentation',
 u'run_id',
 u'fit_filters',
 u'segment_id',
 u'dep_var',
 u'variables',
 u'tscores',
 u'r2',
 u'created',
 u'region',
 u'selected',
 u'observations_name',
 u'yaml',
 u'model_type',
 u'creator',
 u'model_name']

In [79]:
model['observations_name']

u'zones'

In [84]:

yaml.load(model['yaml'])['choice_column']

'zone_id'

In [3]:
%%time
results = list(query.fetch())

CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 990 ms


In [4]:
results[0]['run_id']

u'70318175719s0z'

In [5]:
query.filters

[(u'run_id', '=', u'70318175719s0z')]

In [4]:
len(results)

17

In [21]:
def list_tasks(client):
    query = client.query(kind='Spec')
    query.add_filter('selected', '=', True)
    query.add_filter('model_type', '=', 'regression')
    return list(query.fetch())

In [22]:
selected = list_tasks(client)

In [23]:
len(selected)

3

In [28]:
selected[2]

<Entity(u'Spec', 5740532424769536L) {u'segmentation': u'all_zones', u'run_id': u'70320123805k9u', u'variables': [u'x', u'm2_per_job', u'vacant_residential_units'], u'dep_var': u'np.log1p(avg_residential_value)', u'tscores': <Entity(u'Tscores',) {u'x': -4.591068751049374, u'vacant_residential_units': -2.0155107436500077, u'm2_per_job': -3.404322113387526, u'Intercept': -4.301312681017031}>, u'r2': 0.11118497503951041, u'segment_id': '1', u'region': u'sandiego_zone', u'selected': True, u'observations_name': u'zones', u'created': datetime.datetime(2017, 3, 20, 19, 39, 53, 707425, tzinfo=<UTC>), u'yaml': u'name: RegressionModel\n\nmodel_type: regression\n\nfit_filters:\n- avg_residential_value > 0\n\npredict_filters: all_zones == 1\n\nmodel_expression:\n    left_side: np.log1p(avg_residential_value)\n    right_side:\n    - x\n    - m2_per_job\n    - vacant_residential_units\n\nytransform: np.exp\n\nfitted: true\n\nfit_parameters:\n    Coefficient:\n        Intercept: -78.21502522657191\n  

In [4]:
client = datastore_client

In [3]:
datastore_client.project

u'swarm-test-1470707908646'

In [5]:
kind = "Task"

task_key = datastore_client.key(kind)

task = datastore.Entity(key=task_key)

task['description'] = 'Buy milk'


In [13]:
datastore_client.put(task)


In [1]:
import datetime

In [3]:
a = datetime.datetime.utcnow()

In [8]:
str(a)

'2017-03-17 21:06:18.571389'

In [5]:
import datetime
key = client.key('Task')
task = datastore.Entity(
    key, exclude_from_indexes=['description'])

task.update({
    'created': datetime.datetime.utcnow(),
    'description': 'blah blah',
    'done': False
})

In [7]:
client.put(task)

In [9]:
task

<Entity('Task', 5632018196987904L) {'done': False, 'description': 'blah blah', 'created': datetime.datetime(2017, 3, 20, 1, 33, 25, 558358)}>

In [22]:
task.key.id

5632018196987904L

In [20]:
key = client.key('Task', task.key.id)
client.get(key)['created']

datetime.datetime(2017, 3, 20, 1, 33, 25, 558358, tzinfo=<UTC>)

In [16]:
def add_task(client, description):
    key = client.key('Task')

    task = datastore.Entity(
        key, exclude_from_indexes=['description'])

    task.update({
        'created': datetime.datetime.utcnow(),
        'description': description,
        'done': False
    })

    client.put(task)

    return task.key

In [20]:
task_id = add_task(datastore_client, 'afasfasfasfasfadsfa yoyoyo eddie')

In [18]:
def mark_done(client, task_id):
    with client.transaction():
        key = client.key('Task', task_id)
        task = client.get(key)

        if not task:
            raise ValueError(
                'Task {} does not exist.'.format(task_id))

        task['done'] = True

        client.put(task)

In [23]:
task_id.id

5659313586569216L

In [24]:
mark_done(datastore_client, task_id.id)

In [2]:
def list_tasks(client):
    query = client.query(kind='Task')
    query.order = ['created']

    return list(query.fetch())

In [3]:
list_tasks(datastore_client)

[<Entity(u'Task', 5629499534213120L) {u'done': False, u'description': u'Learn Google Cloud Datastore', u'created': datetime.datetime(2017, 3, 16, 21, 21, 13, 994000, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5649391675244544L) {u'randtext': u'yoyoyo autospec yoyoyo', u'done': False, u'description': u'', u'created': datetime.datetime(2017, 3, 16, 21, 58, 8, 348000, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5639445604728832L) {u'done': False, u'description': 'yoyoyo987654321', u'created': datetime.datetime(2017, 3, 16, 22, 54, 54, 409101, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5707702298738688L) {u'done': False, u'description': 'afasfasfasfasfadsfa yoyoyo eddie', u'created': datetime.datetime(2017, 3, 16, 22, 58, 11, 711875, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5659313586569216L) {u'done': True, u'description': 'afasfasfasfasfadsfa yoyoyo eddie', u'created': datetime.datetime(2017, 3, 16, 23, 0, 17, 68859, tzinfo=<UTC>)}>]

In [4]:
list_tasks(datastore_client)

[<Entity(u'Task', 5629499534213120L) {u'done': False, u'description': u'Learn Google Cloud Datastore', u'created': datetime.datetime(2017, 3, 16, 21, 21, 13, 994000, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5649391675244544L) {u'randtext': u'yoyoyo autospec yoyoyo', u'done': False, u'description': u'', u'created': datetime.datetime(2017, 3, 16, 21, 58, 8, 348000, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5639445604728832L) {u'done': False, u'description': 'yoyoyo987654321', u'created': datetime.datetime(2017, 3, 16, 22, 54, 54, 409101, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5707702298738688L) {u'done': False, u'description': 'afasfasfasfasfadsfa yoyoyo eddie', u'created': datetime.datetime(2017, 3, 16, 22, 58, 11, 711875, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5659313586569216L) {u'done': True, u'description': 'afasfasfasfasfadsfa yoyoyo eddie', u'created': datetime.datetime(2017, 3, 16, 23, 0, 17, 68859, tzinfo=<UTC>)}>]

In [7]:
task = datastore.Entity(client.key('Task'))
task.update({
    'category': 'Personal',
    'done': False,
    'priority': 4,
    'description': 'Learn Cloud Datastore'
})

In [8]:
client.put(task)

In [9]:
print('Saved {}: {}'.format(task.key.name, task['description']))

Saved None: Learn Cloud Datastore


In [5]:
complete_key = client.key('Task', 'sample_task')

task = datastore.Entity(key=complete_key)

task.update({
    'category': 'Personal',
    'done': False,
    'priority': 4,
    'description': 'Learn Cloud Datastore'
})

client.put(task)

In [11]:
key = client.key('Task', 'sample_task')
task = client.get(key)
print task

<Entity(u'Task', u'sample_task') {u'priority': 4L, u'category': 'Personal', u'done': False, u'description': 'Learn Cloud Datastore'}>


In [17]:
def create_task(name, description):
    complete_key = client.key('Task', name)
    task = datastore.Entity(key=complete_key)

    task.update({
        'category': 'Personal',
        'done': False,
        'priority': 1,
        'description': description
    })
    return task

In [18]:
task1 = create_task('yoyo1', 'dfajdfljdklfja yoyoy -1')
task2 = create_task('yoyo2', 'dfajdfljdklfja yoyoy -2')

In [19]:
client.put_multi([task1, task2])

In [26]:
tasks = client.get_multi([task1.key, task2.key])

In [27]:
tasks

[<Entity(u'Task', u'yoyo1') {u'category': 'Personal', u'priority': 1L, u'done': False, u'description': 'dfajdfljdklfja yoyoy -1'}>,
 <Entity(u'Task', u'yoyo2') {u'priority': 1L, u'category': 'Personal', u'done': False, u'description': 'dfajdfljdklfja yoyoy -2'}>]

In [28]:
client.get_multi??

In [7]:
complete_key = client.key('Task')

task = datastore.Entity(key=complete_key)

task.update({
    'category': 'Personal',
    'done': False,
    'priority': 99,
    'description': 'Learn Cloud Datastore'
})

client.put(task)

In [8]:
complete_key = client.key('Task')

task = datastore.Entity(key=complete_key)

task.update({
    'category': 'Personal',
    'done': False,
    'coefs': [1, 2, 3, 4, 5],
    'description': 'Learn Cloud Datastore'
})

client.put(task)

In [12]:
complete_key = client.key('Task')

task = datastore.Entity(key=complete_key)

task.update({
    'category': 'Personal',
    'done': False,
    'result1':1.3,
    'description': 'Learn Cloud Datastore'
})

client.put(task)

In [13]:
complete_key = client.key('Task')

task = datastore.Entity(key=complete_key)

task.update({
    'category': 'Personal',
    'done': False,
    'result1':2.9,
    'result2':5.3,
    'description': 'Learn Cloud Datastore'
})

client.put(task)

In [18]:
%%time
complete_key = client.key('Task')

task = datastore.Entity(key=complete_key)

task.update({
    'result1':3.1,
    'result2':2.0,
    'result3':6.1,
})

client.put(task)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 350 ms


In [19]:
coefficients = {'a':1.1, 'b':2.1, 'c':3.1}
tscores = {'a':4.1, 'b':5.1, 'c':6.1}

In [20]:
key = client.key('Task')

task = datastore.Entity(key=key)

task.update({
    'fit':3.1,
})


In [22]:
embedded_key = client.key('Coef')
embedded_entity = datastore.Entity(key=embedded_key)
embedded_entity['result1'] = 8.2
embedded_entity['result2'] = 2.3
## todo:  give embedded entity the proposal_id
## todo:  update with a dictionary of coefs all at once
## todo:  look into ancestors as an alternative to embedded entities, but can this be done in one request?
task['coefficients']=embedded_entity
client.put(task)

In [24]:
key = client.key('Task', 5715999101812736)
task = client.get(key)
print task

<Entity(u'Task', 5715999101812736L) {u'coefficients': <Entity(u'Coef',) {u'result2': 2.3, u'result1': 8.2}>, u'fit': 3.1}>


In [27]:
coefs = task['coefficients']

In [34]:
coefs

<Entity(u'Coef',) {u'result2': 2.3, u'result1': 8.2}>

In [30]:
coefs.keys()

[u'result2', u'result1']

In [32]:
coefs['result1']

8.2

In [33]:
coefs.has_key('result2')

True

In [37]:
query = client.query(kind='Task')
list(query.fetch())

[<Entity(u'Task', 5629499534213120L) {u'done': False, u'description': u'Learn Google Cloud Datastore', u'created': datetime.datetime(2017, 3, 16, 21, 21, 13, 994000, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5631986051842048L) {u'result2': 2.0, u'result3': 6.1, u'result1': 3.1}>,
 <Entity(u'Task', 5634472569470976L) {u'priority': 4L, u'category': 'Personal', u'done': False, u'description': 'Learn Cloud Datastore'}>,
 <Entity(u'Task', 5639445604728832L) {u'done': False, u'description': 'yoyoyo987654321', u'created': datetime.datetime(2017, 3, 16, 22, 54, 54, 409101, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5644406560391168L) {u'category': 'Personal', u'done': False, u'description': 'Learn Cloud Datastore', u'coefs': [1L, 2L, 3L, 4L, 5L]}>,
 <Entity(u'Task', 5649391675244544L) {u'randtext': u'yoyoyo autospec yoyoyo', u'done': False, u'description': u'', u'created': datetime.datetime(2017, 3, 16, 21, 58, 8, 348000, tzinfo=<UTC>)}>,
 <Entity(u'Task', 5654313976201216L) {u'priority': 99L, u'category': '

In [38]:
query = client.query(kind='Coef')
list(query.fetch())

[]

In [1]:
spec = ['avg_residential_value', 'avg_year_built']

llr = 0.05234607777386069

tscores = {'avg_residential_value': -5.391276176860389, 'avg_year_built': -1294.0203966397366}

In [2]:
str(spec)

"['avg_residential_value', 'avg_year_built']"

In [8]:
%%time
key = client.key('Spec')
ds_spec = datastore.Entity(key=key)
ds_spec['variables'] = spec
ds_spec['llr'] = llr

sub_key = client.key('Tscores')
ds_tscores = datastore.Entity(key=sub_key)
ds_tscores.update(tscores)
ds_spec['tscores']=ds_tscores
client.put(ds_spec)


## todo:  give embedded entity the proposal_id
## todo:  update with a dictionary of coefs all at once
## todo:  look into ancestors as an alternative to embedded entities, but can this be done in one request?

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 223 ms
